# Linegraph - Example App

In [1]:
import sys
import pathlib
import subprocess
from pydantic import validator
from IPython.display import display

from ipyautoui import AutoUi
from ipyautoui import AutoDisplay
from ipyautoui.autodisplayfile_renderers import display_python_file

sys.path.append(
    str(pathlib.Path("../../../src").resolve())
)  # append ipyrun. TODO: for dev only. remove at build time.


from ipyrun import RunApp, BatchApp, ConfigShell, DefaultConfigShell, RunShellActions
from ipyrun.runshell import (
    AutoDisplayDefinition,
    create_autodisplay_map,
    ConfigBatch,
    DefaultConfigShell,
    DefaultBatchActions,
    BatchShellActions,
)


DIR_CORE = pathlib.Path("./linegraph_core").resolve()
PATH_RUN = DIR_CORE #/ "script_linegraph.py"
FPTH_INPUTS_SCHEMA = DIR_CORE / "input_schema_linegraph.py"
DIR_APPDATA = pathlib.Path("./linegraph_appdata").resolve()

# extend DefaultConfigShell to LineGraphConfigShell


class LineGraphConfigShell(DefaultConfigShell):
    
    @validator("update_config_at_runtime", always=True, pre=True)
    def _update_config_at_runtime(v):
        return False
    
    @validator("path_run", always=True, pre=True)
    def _set_path_run(cls, v, values):
        return PATH_RUN

    @validator("fpths_outputs", always=True)
    def _fpths_outputs(cls, v, values):
        fdir = values["fdir_appdata"]
        key = values["key"]
        paths = [
            fdir / ("out-" + key + ".csv"),
            fdir / ("out-" + key + ".plotly.json"),
        ]
        return paths

    @validator("autodisplay_definitions", always=True)
    def _autodisplay_definitions(cls, v, values):
        return [
            AutoDisplayDefinition(
                path=FPTH_INPUTS_SCHEMA,
                obj_name="LineGraph",
                ext=".lg.json",
                ftype="in",
            )
        ]

    @validator("autodisplay_inputs_kwargs", always=True)
    def _autodisplay_inputs_kwargs(cls, v, values):
        return dict(patterns="*")

    @validator("autodisplay_outputs_kwargs", always=True)
    def _autodisplay_outputs_kwargs(cls, v, values):
        return dict(patterns="*.plotly.json")


# extend ConfigBatch to LineGraphConfigBatch


class LineGraphConfigBatch(ConfigBatch):
    @validator("cls_config", always=True)
    def _cls_config(cls, v, values):
        """bundles RunApp up as a single argument callable"""
        return LineGraphConfigShell


class LineGraphBatchActions(BatchShellActions):

    @validator("runlog_show", always=True)
    def _runlog_show(cls, v, values):
        return None


config_batch = LineGraphConfigBatch(
    fdir_root=DIR_APPDATA,
    # cls_config=MyConfigShell,
    title="""# Plot Straight Lines\n### example RunApp""",
)
if config_batch.fpth_config.is_file():
    config_batch = LineGraphConfigBatch.parse_file(config_batch.fpth_config)
app = BatchApp(config_batch, cls_actions=LineGraphBatchActions)
display(app)

BatchApp(children=(VBox(children=(HTML(value='<h1>Plot Straight Lines</h1>\n<h3>example RunApp</h3>'), HBox(ch…